In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [176]:
def norm(data, norm_data):
    train_stats = data.describe()
    train_stats = train_stats.transpose()
    return(norm_data - train_stats['mean'])// train_stats['std']

In [177]:
data = pd.read_csv("u.data",   delimiter='\t'   )
data.columns = ['UserID','itemID','rating','timestamp']
data = norm(data, data)
all_users = data['UserID']
users = np.unique(all_users)
users = list(users)
items_all = data['itemID']

In [178]:
def data_processing (matrix, data):
    for i in range(0,data.shape[0]):
        userID = data.iloc[i,0]
        itemID = data.iloc[i,1]
        rating = data.iloc[i,2]
        matrix[userID-1,itemID-1] = rating
    matrix = pd.DataFrame(matrix)
    matrix.to_csv('user_item_data.csv', sep=';', encoding='utf-8') 

In [179]:
items = np.unique(items_all)
items = list(items)

matrix_df = pd.DataFrame(0,index = users, columns = items)

train_data, test_data = np.split(data.sample(frac=1), [int(.8*len(data))])

In [180]:
data_processing(matrix_df.as_matrix(),train_data)
   

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [139]:
train_data.to_csv('train_data.csv', index=False , sep=';', encoding='utf-8') 

test_data.to_csv('test_data.csv', index=False, sep=';', encoding='utf-8') 

In [140]:
Rating = pd.read_csv("user_item_data.csv",   delimiter=';'   )
Rating.drop(Rating.columns[0],axis=1 ,inplace=True)
train_data = pd.read_csv("train_data.csv",   delimiter=';'   )

test_data = pd.read_csv("test_data.csv",   delimiter=';'   )

In [141]:
Rating.head()

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0,3,0,3,0,5,0,1,5,0,...,0,0,0,0,0,0,0,0,0,0
1,4,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [142]:
train_data.head()

,UserID,itemID,rating,timestamp
0,504,238,3,887912416
1,190,100,4,891033653
2,707,529,4,886287376
3,932,1451,5,891249675
4,747,1204,4,888639102


In [143]:
def matrix_factorization(Rating, P, Q, comp, alpha,lamda,steps):
    for step in range(0,steps):
        for i in range(0,len(Rating)):
            for j in range(0,len(Rating[i])):
                if(Rating[i][j] > 0) :
                    
                    T = Q.T
                    ij = Rating[i][j] - np.dot(P[i,:],T[:,j])
                    for C in range(0,comp):
                        P[i][C] = P[i][C] + alpha * ( 2* (ij * Q[j][C] - lamda * P[i][C]))
                        Q[j][C] = Q[j][C] + alpha * ( 2* (ij * P[i][C] - lamda * Q[j][C]))
    return P, Q.T

In [144]:
def RMSE(matrix,data):
    matrix = matrix.as_matrix()
    data.columns = ['userID','itemID','rating','timeStamp']

    df_rating = np.array(data.iloc[:,2])
    print('Data to be rated: ',df_rating.shape)
    cont = []
    for i in range(0,data.shape[0]):
     
        ff = data.ix[i,0]
        hh= data.ix[i,1]  
        md = matrix[ff-1,hh-1]
        cont.append(md)
    cont = np.array(cont) 
    cont = cont - df_rating
    cont = (cont)**2    
    out = np.sqrt(sum(cont)/data.shape[0])
    print('RMSE',out)    
    return out

In [145]:
Rating = np.array(Rating)    
logfile = []
resultRmse=[]
alpha1 = [np.arange(0.0, 3, 0.01)]
lamda1 = [np.arange(0.0,3,0.01)]
alp = 0.01
lam = 0.01

In [101]:
for alp1,lam1 in zip(alpha1[0],lamda1[0]):
    for i in range(1,100):
        col = len(Rating)
        row = len(Rating[0])   
        K = i
        P = np.random.rand(col,K)
        Q = np.random.rand(row,K)
        nP, nQ = matrix_factorization(Rating, P, Q, K, alp,lam,1)
        nR = np.dot(nP, nQ)
        nR = pd.DataFrame(nR)
        train_score = RMSE(nR,train_data)
        
        test_score = RMSE(nR,test_data)
        resultRmse.append([train_score,test_score])
        x = 'Hyper Parameter: '+'alpha:'+str(alp)+ 'lamda:'+ str(lam)+' latent : '+ str(i)+' train_Score:'+str(train_score)+' test_Score:'+str(test_score)    
        print(x)

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.
C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()


Data to be rated:  (79999,)
result  [ 5.52357851  5.93321131  6.03501093 ... 10.57123614  2.62465728
  1.47272553]
RMSE 3.3501581985534523
Data to be rated:  (20000,)
result  [4.64565859 5.48997743 6.24928551 ... 1.77308471 6.09364782 6.2717709 ]
RMSE 3.385979836684825
Hyper Parameter: alpha:0.01lamda:0.01 latent : 1 train_Score:3.3501581985534523 test_Score:3.385979836684825
Data to be rated:  (79999,)


KeyboardInterrupt: 

In [147]:
alp = 0.01
lam = 0.01
col = len(Rating)
row = len(Rating[0])   
K = 30
P = np.random.rand(col,K)
Q = np.random.rand(row,K)
nP, nQ = matrix_factorization(Rating, P, Q, K, alp,lam,10)
nR = np.dot(nP, nQ)
nR = pd.DataFrame(nR)
train_score = RMSE(nR,train_data)
        
test_score = RMSE(nR,test_data)
resultRmse.append([train_score,test_score])
x = 'Hyper Parameter: '+'alpha:'+str(alp)+ 'lamda:'+ str(lam)+' latent : '+ str(K)+' train_Score:'+str(train_score)+' test_Score:'+str(test_score)    
print(x)

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.
C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()


Data to be rated:  (79999,)
RMSE 0.6662996309803008
Data to be rated:  (20000,)
RMSE 1.1009910633822755
Hyper Parameter: alpha:0.01lamda:0.01 latent : 30 train_Score:0.6662996309803008 test_Score:1.1009910633822755
